## Süntaksianalüüs

Selles notebookis kasutatakse ära automaatset süntaksianalüüsi, et leida huvipakkuvate nimeüksustega seotud täiendid (millised nad on), verbid (mida nad teevad) ning nende verbide küljes olevad määrused (kuidas midagi tehakse jne). Leitud sõnade põhjal pannakse paika üksikud laiemad teemad ning valitakse sõnade seast teemasid ilmestavad märksõnad, mille kaudu saab lauseid temaatilistesse klassidesse jaotada.

In [1]:
from estnltk import Text
from estnltk.taggers import StanzaSyntaxEnsembleWebTagger
from estnltk_neural.taggers.syntax.stanza_tagger.ensemble_tagger import StanzaSyntaxEnsembleTagger
from estnltk_neural.taggers import StanzaSyntaxTagger
from estnltk.taggers import SyntaxDependencyRetagger
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
import csv
from collections import Counter

### I taggerid

Veebi kaudu kasutatav tagger:

In [11]:
stanza_syntax_ensemble_web_tagger = \
    StanzaSyntaxEnsembleWebTagger(url='https://api.tartunlp.ai/estnltk/tagger/stanza_syntax')
stanza_syntax_ensemble_web_tagger

StanzaSyntaxEnsembleWebTagger(input_layers=('words', 'sentences', 'morph_extended'), output_layer=stanza_ensemble_syntax, output_attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'), url=https://api.tartunlp.ai/estnltk/tagger/stanza_syntax, batch_layer=words, batch_max_size=125, batch_enveloping_layer=sentences)

Siin võetakse märgendamisel kasutusele Stanza ensemble taggeri asemel tavaline Stanza tagger, sest selle töökiirus on parem.

In [3]:
stanza_tagger = StanzaSyntaxTagger(input_type='morph_analysis', input_morph_layer='morph_analysis', add_parent_and_children=True)

### II andmete sisselugemine ja süntaksianalüüsi kihi peale märgendamine

Kõigepealt loetakse andmebaasist sisse laused, millele on PhraseTaggeriga märgendatud peale mõni huvipakkuvatest nimeüksustest.

In [4]:
sentences = []
sentence_ids = []

In [5]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [6]:
# ID asemel võtta tabelist sentence_ID
for row in cur.execute("SELECT sentence_ID, sentence FROM phrase_tagger_sentences"):
    sentence_id = row[0]
    sentence = json_to_text(json_text=row[1])
    sentences.append(sentence)
    sentence_ids.append(sentence_id)

In [7]:
con.close()

In [8]:
len(sentences)

18569

Seejärel märgendatakse lausetele süntaksianalüüsi kiht.

In [18]:
#Veebitaggeriga
for sentence in sentences:
    sentence.tag_layer('morph_extended')
    stanza_syntax_ensemble_web_tagger.tag(sentence)
    SyntaxDependencyRetagger('stanza_ensemble_syntax').retag(sentence) #Parent ja children atribuutide lisamiseks kihile

NotImplementedError: ('ambiguous enveloping layer: entity_phrases', "in the 'StanzaSyntaxEnsembleWebTagger'")

Veebitaggeriga tekkis märgendamisel mõne lause juures viga, mis on seotud PhraseTaggeri abil peale märgendatud nimeüksuste kihiga.

In [10]:
for sentence in sentences:
    stanza_tagger.tag(sentence)

RuntimeError: ('[enforce fail at ..\\c10\\core\\impl\\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6370721952 bytes.', "in the 'StanzaSyntaxTagger'")